# 1. Capital Gains Tracker
The `CapGainsTracker` processes buy and sell trades in chronological order.  It calculates capital gains in accordance with 2025 IRS rules on:
1. **Term:** Whether a gain is Short-Term (ST, taxed as income) or Long-Term (LT, held for more than 1 year and taxed at lower capital-gain rates).
2. **Wash sales:** When a capital loss is realized, it can be "washed" by "replacement" trades made in the same asset within 30 days before or after realization.  When washed, a loss can't be immediately claimed for tax purposes and must instead be added to the basis of the replacement shares.

*Limitations:*
1. This is done to the best of our ability to understand/interpret [IRS Publication 550](https://www.irs.gov/pub/irs-pdf/p550.pdf).
1. This does not handle accounting of derivatives.
2. This does not look for, or account for, "substantially identical" assets.  If you enter trades for asset `XYZ`, and also for `XYZ.B`, it treats them as separate even if the IRS would consider them "substantially identical."

In [1]:
import pandas as pd
from tax_tracker import CapGainsTracker

## Example 1.1
A series of trades for shares of hypothetical asset `ABC` that illustrate how losses are washed by replacement trades made both before and after the loss is realized.  Prices and dates are made up for this example.

We'll begin by buying 10 shares, then selling 5 of them at a price $1/share lower, which gives us a Short-Term capital loss of $5 and leaves an open position of 5 shares.

In [2]:
tracker = CapGainsTracker()
# Buy 10 shares of ABC at $10
start_date = pd.Timestamp('2022-01-05')
tracker.trade(start_date, 'ABC', shares=10, price=10)
# Sell 5 shares of ABC at $9, realizing capital loss of $1/share
sell_date = start_date + pd.Timedelta(days=1)
tracker.trade(sell_date, 'ABC', -5, price=9)
assert tracker.capital_gains[sell_date]['ShortTerm'] == -5.0, "Realized -$5 ST gain"
display(tracker.capital_gains_df)
print("Closed position:\n"+tracker.closed_lots_str)
print("\nOpen position:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2022-01-06,-5.0,0


Closed position:
ABC: 2022-01-05 5 @ $10.00 >> 2022-01-06 -5 @ $9.00 = $-5.00 ST (1 day)

Open position:
ABC: 5 @ $10.00 (2022-01-05) 


Now the madness begins.  Within 30 days of that loss sale we buy back 5 shares, which washes the loss: It will no longer appear in the Capital Gains ledger, and instead the loss will be added to the basis of the "replacement" shares we bought.

In [3]:
# Buy 5 shares of ABC within 30 days to trigger wash of the Short Term loss
buy_back_date = sell_date + pd.Timedelta(days=25)
tracker.buy(buy_back_date, 'ABC', shares=5, price=10)

# Show how this changed our accounting
display(tracker.capital_gains_df)
assert tracker.capital_gains[sell_date]['ShortTerm'] == 0, "Capital loss washed"
# We can check the list of wash sales
print("Wash sales:\n"+tracker.washed_lots_str)
wash_sale = tracker.wash_trades[0]
assert wash_sale.washed_quantity == 5, "Wash sale quantity"
# Verify basis added to replacement shares
replacement = tracker.positions['ABC'][-1]
assert replacement.basis_add == -5, "Replacement basis"
assert replacement.shares == 5, "Replacement shares"
assert replacement.basis_days == (sell_date - start_date).days, "Replacement basis days"
print(f"\nOpen positions after repurchase on {buy_back_date.date()}:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2022-01-06,0.0,0


Wash sales:
ABC: 2022-01-05 5 @ $10.00 >> 2022-01-06 -5 @ $9.00 = $-5.00 ST (1 day) washed on 2022-01-31

Open positions after repurchase on 2022-01-31:
ABC: 5 @ $10.00 (2022-01-05) 
ABC: 5 @ $10.00 (2022-01-31)  Basis $-5.00 term+1 day


We'll hold those positions until just over a year, then sell them at a loss.  The original 5 shares produces a Long-Term capital loss, but the replacement shares haven't been held for a full year so their capital loss is classified as Short-Term.

In [4]:
# Just over a year later, sell all shares
second_sell_date = start_date + pd.DateOffset(days=367)
tracker.sell(second_sell_date, 'ABC', shares_to_sell=10, price=8)
print(tracker.open_lots_str)
display(tracker.capital_gains_df)
print("Closed position records:\n"+tracker.closed_lots_str)

No positions


,ShortTerm,LongTerm
2022-01-06,0.0,0.0
2023-01-07,-15.0,-10.0


Closed position records:
ABC: 2022-01-05 5 @ $10.00 >> 2022-01-06 -5 @ $9.00 = $-5.00 ST (1 day) washed
ABC: 2022-01-05 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-10.00 LT (367 days)
ABC: 2022-01-31 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-15.00 ST (341+1 days)


Let's complicate it again: Buying 10 shares within 30 days of that sale again triggers the wash rule for the capital losses, so they will disappear from our Capital Gains ledger.  Also, this single purchase produces two different tax lots because there are two different washed losses being carried along.

In [5]:
# Buy 10 shares back within 30 days to trigger wash sale
second_buy_back_date = second_sell_date + pd.Timedelta(days=25)
tracker.buy(second_buy_back_date, 'ABC', shares=10, price=7)
print("Wash sales:\n"+tracker.washed_lots_str)
display(tracker.capital_gains_df)
print(f"Open positions after repurchase on {second_buy_back_date.date()}:\n"+tracker.open_lots_str)

Wash sales:
ABC: 2022-01-05 5 @ $10.00 >> 2022-01-06 -5 @ $9.00 = $-5.00 ST (1 day) washed on 2022-01-31
ABC: 2022-01-31 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-15.00 ST (341+1 days) washed on 2023-02-01
ABC: 2022-01-05 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-10.00 LT (367 days) washed on 2023-02-01


,ShortTerm,LongTerm
2022-01-06,0.0,0.0
2023-01-07,0.0,0.0


Open positions after repurchase on 2023-02-01:
ABC: 5 @ $7.00 (2023-02-01)  Basis $-15.00 term+342 days
ABC: 5 @ $7.00 (2023-02-01)  Basis $-10.00 term+367 days


Now let's sell everything and see how the accounting finishes.

In [6]:
date_out = second_buy_back_date + pd.Timedelta(days=1)
tracker.sell(date_out, 'ABC', shares_to_sell=10, price=8.5)
print("After final sale: "+tracker.open_lots_str)
display(tracker.capital_gains_df)
print("Trade record:\n"+tracker.closed_lots_str)

After final sale: No positions


,ShortTerm,LongTerm
2022-01-06,0.0,0.0
2023-01-07,0.0,0.0
2023-02-02,-7.5,-2.5


Trade record:
ABC: 2022-01-05 5 @ $10.00 >> 2022-01-06 -5 @ $9.00 = $-5.00 ST (1 day) washed
ABC: 2022-01-05 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-10.00 LT (367 days) washed
ABC: 2022-01-31 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-15.00 ST (341+1 days) washed
ABC: 2023-02-01 5 @ $7.00 >> 2023-02-02 -5 @ $8.50 = $-7.50 ST (1+342 days)
ABC: 2023-02-01 5 @ $7.00 >> 2023-02-02 -5 @ $8.50 = $-2.50 LT (1+367 days)


## Example 1.2
We'll repeat the same trade sequence from Example 1.1, but we'll space the transactions so that more than 30 days elapses between each.  This avoids the wash rule.

In [7]:
tracker = CapGainsTracker()
# Buy 10 shares of ABC at $10
start_date = pd.Timestamp('2022-01-05')
tracker.buy(start_date, 'ABC', shares=10, price=10)
# Sell 5 shares of ABC at $9, realizing capital loss of $1/share
sell_date = start_date + pd.Timedelta(days=1)
tracker.sell(sell_date, 'ABC', shares_to_sell=5, price=9)
assert tracker.capital_gains[sell_date]['ShortTerm'] == -5.0, "Realized -$5 ST gain"
print("Closed position:\n"+tracker.closed_lots_str)
display(tracker.capital_gains_df)
print("Open position:\n"+tracker.open_lots_str)

Closed position:
ABC: 2022-01-05 5 @ $10.00 >> 2022-01-06 -5 @ $9.00 = $-5.00 ST (1 day)


,ShortTerm,LongTerm
2022-01-06,-5.0,0


Open position:
ABC: 5 @ $10.00 (2022-01-05) 


In [8]:
# Buy 5 shares of ABC more than 30 days later
buy_back_date = sell_date + pd.Timedelta(days=35)
tracker.buy(buy_back_date, 'ABC', shares=5, price=10)
display(tracker.capital_gains_df)
print("Open position:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2022-01-06,-5.0,0


Open position:
ABC: 5 @ $10.00 (2022-01-05) 
ABC: 5 @ $10.00 (2022-02-10) 


In [9]:
# Just over a year later, sell all shares
second_sell_date = start_date + pd.DateOffset(days=367)
tracker.sell(second_sell_date, 'ABC', shares_to_sell=10, price=8)
print(tracker.open_lots_str)
display(tracker.capital_gains_df)
print("\nClosed position records:\n"+tracker.closed_lots_str)

No positions


,ShortTerm,LongTerm
2022-01-06,-5.0,0.0
2023-01-07,-10.0,-10.0



Closed position records:
ABC: 2022-01-05 5 @ $10.00 >> 2022-01-06 -5 @ $9.00 = $-5.00 ST (1 day)
ABC: 2022-01-05 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-10.00 LT (367 days)
ABC: 2022-02-10 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-10.00 ST (331 days)


In [10]:
# Buy 10 shares back more than 30 days later
second_buy_back_date = second_sell_date + pd.Timedelta(days=35)
tracker.buy(second_buy_back_date, 'ABC', shares=10, price=7)
print(tracker.open_lots_str)
display(tracker.capital_gains_df)

ABC: 10 @ $7.00 (2023-02-11) 


,ShortTerm,LongTerm
2022-01-06,-5.0,0.0
2023-01-07,-10.0,-10.0


In [11]:
date_out = second_buy_back_date + pd.Timedelta(days=1)
tracker.sell(date_out, 'ABC', shares_to_sell=10, price=8.5)
print("After final sale: "+tracker.open_lots_str)
display(tracker.capital_gains_df)
print("Trade record:\n"+tracker.closed_lots_str)

After final sale: No positions


,ShortTerm,LongTerm
2022-01-06,-5.0,0.0
2023-01-07,-10.0,-10.0
2023-02-12,15.0,0.0


Trade record:
ABC: 2022-01-05 5 @ $10.00 >> 2022-01-06 -5 @ $9.00 = $-5.00 ST (1 day)
ABC: 2022-01-05 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-10.00 LT (367 days)
ABC: 2022-02-10 5 @ $10.00 >> 2023-01-07 -5 @ $8.00 = $-10.00 ST (331 days)
ABC: 2023-02-11 10 @ $7.00 >> 2023-02-12 -10 @ $8.50 = $15.00 ST (1 day)


## Example 1.3
We'll repeat the same trade sequence from Example 1.1, but in the opposite direction (i.e., beginning with a short sale) so that instead of a loss we have a gain.  This also avoids the wash rule.

In [12]:
tracker = CapGainsTracker()
# Short 10 shares of ABC at $10
start_date = pd.Timestamp('2022-01-05')
tracker.trade(start_date, 'ABC', shares=-10, price=10)
# Cover 5 shares of ABC at $9, realizing capital gain of $1/share
cover_date = start_date + pd.Timedelta(days=1)
tracker.trade(cover_date, 'ABC', 5, price=9)
assert tracker.capital_gains[cover_date]['ShortTerm'] == 5.0, "Realized $5 ST gain"
display(tracker.capital_gains_df)
print("Closed position:\n"+tracker.closed_lots_str)
print("\nOpen position:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2022-01-06,5.0,0


Closed position:
ABC: 2022-01-05 -5 @ $10.00 >> 2022-01-06 5 @ $9.00 = $5.00 ST (1 day)

Open position:
ABC: -5 @ $10.00 (2022-01-05) 


In [13]:
# Short 5 shares of ABC within 30 days; no effect on capital gains
second_short_date = cover_date + pd.Timedelta(days=25)
tracker.trade(second_short_date, 'ABC', shares=-5, price=10)
display(tracker.capital_gains_df)
print("Open positions:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2022-01-06,5.0,0


Open positions:
ABC: -5 @ $10.00 (2022-01-05) 
ABC: -5 @ $10.00 (2022-01-31) 


In [14]:
# Just over a year later, cover all shares
second_cover_date = start_date + pd.DateOffset(days=367)
tracker.trade(second_cover_date, 'ABC', 10, price=8)
print(tracker.open_lots_str)
display(tracker.capital_gains_df)
print("Closed position records:\n"+tracker.closed_lots_str)

No positions


,ShortTerm,LongTerm
2022-01-06,5.0,0.0
2023-01-07,10.0,10.0


Closed position records:
ABC: 2022-01-05 -5 @ $10.00 >> 2022-01-06 5 @ $9.00 = $5.00 ST (1 day)
ABC: 2022-01-05 -5 @ $10.00 >> 2023-01-07 5 @ $8.00 = $10.00 LT (367 days)
ABC: 2022-01-31 -5 @ $10.00 >> 2023-01-07 5 @ $8.00 = $10.00 ST (341 days)


In [15]:
# Short 10 shares within 30 days.  Still no loss, so no wash
second_short_date = second_cover_date + pd.Timedelta(days=25)
tracker.trade(second_short_date, 'ABC', -10, price=7)
display(tracker.capital_gains_df)
print(f"Open positions after shorting on {second_short_date.date()}:\n"+tracker.open_lots_str)
print("\nWash sales: " + tracker.washed_lots_str)

,ShortTerm,LongTerm
2022-01-06,5.0,0.0
2023-01-07,10.0,10.0


Open positions after shorting on 2023-02-01:
ABC: -10 @ $7.00 (2023-02-01) 

Wash sales: No washed trades


In [16]:
# Sell everything and check final accounting
date_out = second_short_date + pd.Timedelta(days=1)
tracker.trade(date_out, 'ABC', 10, price=8.5)
print("After final cover: "+tracker.open_lots_str)
display(tracker.capital_gains_df)
print("Trade record:\n"+tracker.closed_lots_str)

After final cover: No positions


,ShortTerm,LongTerm
2022-01-06,5.0,0.0
2023-01-07,10.0,10.0
2023-02-02,-15.0,0.0


Trade record:
ABC: 2022-01-05 -5 @ $10.00 >> 2022-01-06 5 @ $9.00 = $5.00 ST (1 day)
ABC: 2022-01-05 -5 @ $10.00 >> 2023-01-07 5 @ $8.00 = $10.00 LT (367 days)
ABC: 2022-01-31 -5 @ $10.00 >> 2023-01-07 5 @ $8.00 = $10.00 ST (341 days)
ABC: 2023-02-01 -10 @ $7.00 >> 2023-02-02 10 @ $8.50 = $-15.00 ST (1 day)


## Example 1.4
Orders that not only wash losses, but also change the position side (from long to short, and vice versa).

In [17]:
tracker = CapGainsTracker()
# Buy 5 shares of ABC at $10
start_date = pd.Timestamp('2022-01-05')
tracker.trade(start_date, 'ABC', shares=5, price=10)
# Sell 3 shares of ABC at $9, realizing capital loss of $1/share
sell_date = start_date + pd.Timedelta(days=1)
tracker.trade(sell_date, 'ABC', -3, price=9)
display(tracker.capital_gains_df)
print("Closed position:\n"+tracker.closed_lots_str)
print("\nOpen position:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2022-01-06,-3.0,0


Closed position:
ABC: 2022-01-05 3 @ $10.00 >> 2022-01-06 -3 @ $9.00 = $-3.00 ST (1 day)

Open position:
ABC: 2 @ $10.00 (2022-01-05) 


In [18]:
# Sell 7 shares of ABC at $8, realizing capital loss of $2/share on 2 shares, but not washing the loss
second_sell_date = start_date + pd.Timedelta(days=2)
tracker.trade(second_sell_date, 'ABC', -7, price=8)
display(tracker.capital_gains_df)
print("Closed positions:\n"+tracker.closed_lots_str)
print("\nOpen position:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2022-01-06,-3.0,0
2022-01-07,-4.0,0


Closed positions:
ABC: 2022-01-05 3 @ $10.00 >> 2022-01-06 -3 @ $9.00 = $-3.00 ST (1 day)
ABC: 2022-01-05 2 @ $10.00 >> 2022-01-07 -2 @ $8.00 = $-4.00 ST (2 days)

Open position:
ABC: -5 @ $8.00 (2022-01-07) 


In [19]:
# Buy 10 shares.  This will wash the losses, and produce a loss of its own.
buy_back_date = second_sell_date + pd.Timedelta(days=25)
tracker.trade(buy_back_date, 'ABC', shares=10, price=10)
print("Wash sales:\n"+tracker.washed_lots_str)
display(tracker.capital_gains_df)
print("Closed positions:\n"+tracker.closed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)

Wash sales:
ABC: 2022-01-05 2 @ $10.00 >> 2022-01-07 -2 @ $8.00 = $-4.00 ST (2 days) washed on 2022-02-01
ABC: 2022-01-05 3 @ $10.00 >> 2022-01-06 -3 @ $9.00 = $-3.00 ST (1 day) washed on 2022-02-01


,ShortTerm,LongTerm
2022-01-06,0.0,0
2022-01-07,0.0,0
2022-02-01,-10.0,0


Closed positions:
ABC: 2022-01-05 3 @ $10.00 >> 2022-01-06 -3 @ $9.00 = $-3.00 ST (1 day) washed
ABC: 2022-01-05 2 @ $10.00 >> 2022-01-07 -2 @ $8.00 = $-4.00 ST (2 days) washed
ABC: 2022-01-07 -5 @ $8.00 >> 2022-02-01 5 @ $10.00 = $-10.00 ST (25 days)

Open positions:
ABC: 2 @ $10.00 (2022-02-01)  Basis $-4.00 term+2 days
ABC: 3 @ $10.00 (2022-02-01)  Basis $-3.00 term+1 day


In [20]:
# Sell 10 shares.  This will wash the current loss but realize the previously washed losses.
sell_date = buy_back_date + pd.Timedelta(days=20)
tracker.trade(sell_date, 'ABC', -10, price=9)
print("Wash sales:\n"+tracker.washed_lots_str)
display(tracker.capital_gains_df)
print("Closed positions:\n"+tracker.closed_lots_str)
print("\nOpen position:\n"+tracker.open_lots_str)

Wash sales:
ABC: 2022-01-05 2 @ $10.00 >> 2022-01-07 -2 @ $8.00 = $-4.00 ST (2 days) washed on 2022-02-01
ABC: 2022-01-05 3 @ $10.00 >> 2022-01-06 -3 @ $9.00 = $-3.00 ST (1 day) washed on 2022-02-01
ABC: 2022-01-07 -5 @ $8.00 >> 2022-02-01 5 @ $10.00 = $-10.00 ST (25 days) washed on 2022-02-21


,ShortTerm,LongTerm
2022-01-06,0.0,0
2022-01-07,0.0,0
2022-02-01,0.0,0
2022-02-21,-12.0,0


Closed positions:
ABC: 2022-01-05 3 @ $10.00 >> 2022-01-06 -3 @ $9.00 = $-3.00 ST (1 day) washed
ABC: 2022-01-05 2 @ $10.00 >> 2022-01-07 -2 @ $8.00 = $-4.00 ST (2 days) washed
ABC: 2022-01-07 -5 @ $8.00 >> 2022-02-01 5 @ $10.00 = $-10.00 ST (25 days) washed
ABC: 2022-02-01 2 @ $10.00 >> 2022-02-21 -2 @ $9.00 = $-6.00 ST (20+2 days)
ABC: 2022-02-01 3 @ $10.00 >> 2022-02-21 -3 @ $9.00 = $-6.00 ST (20+1 days)

Open position:
ABC: -5 @ $9.00 (2022-02-21)  Basis $-10.00 term+25 days


In [21]:
# Sell everything and check final accounting
date_out = sell_date + pd.Timedelta(days=1)
tracker.trade(date_out, 'ABC', 5, price=10)
display(tracker.capital_gains_df)
print("Closed positions:\n"+tracker.closed_lots_str)
print("\nOpen positions: "+tracker.open_lots_str)

,ShortTerm,LongTerm
2022-01-06,0.0,0
2022-01-07,0.0,0
2022-02-01,0.0,0
2022-02-21,-12.0,0
2022-02-22,-15.0,0


Closed positions:
ABC: 2022-01-05 3 @ $10.00 >> 2022-01-06 -3 @ $9.00 = $-3.00 ST (1 day) washed
ABC: 2022-01-05 2 @ $10.00 >> 2022-01-07 -2 @ $8.00 = $-4.00 ST (2 days) washed
ABC: 2022-01-07 -5 @ $8.00 >> 2022-02-01 5 @ $10.00 = $-10.00 ST (25 days) washed
ABC: 2022-02-01 2 @ $10.00 >> 2022-02-21 -2 @ $9.00 = $-6.00 ST (20+2 days)
ABC: 2022-02-01 3 @ $10.00 >> 2022-02-21 -3 @ $9.00 = $-6.00 ST (20+1 days)
ABC: 2022-02-21 -5 @ $9.00 >> 2022-02-22 5 @ $10.00 = $-15.00 ST (1+25 days)

Open positions: No positions


## Example 1.5
Loss is washed over multiple repurchases.

In [22]:
tracker = CapGainsTracker()
tracker.trade(pd.Timestamp('2023-01-15'), 'XYZ', shares=100, price=20)
tracker.trade(pd.Timestamp('2023-02-10'), 'XYZ', shares=-100, price=15)
tracker.trade(pd.Timestamp('2023-02-25'), 'XYZ', shares=40, price=17)
display(tracker.capital_gains_df)
print("Wash sales:\n"+tracker.washed_lots_str)
print("\nClosed positions:\n"+tracker.closed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2023-02-10,-300.0,0


Wash sales:
XYZ: 2023-01-15 40 @ $20.00 >> 2023-02-10 -40 @ $15.00 = $-200.00 ST (26 days) washed on 2023-02-25

Closed positions:
XYZ: 2023-01-15 40 @ $20.00 >> 2023-02-10 -40 @ $15.00 = $-200.00 ST (26 days) washed
XYZ: 2023-01-15 60 @ $20.00 >> 2023-02-10 -60 @ $15.00 = $-300.00 ST (26 days)

Open positions:
XYZ: 40 @ $17.00 (2023-02-25)  Basis $-200.00 term+26 days


In [23]:
tracker.trade(pd.Timestamp('2023-03-08'), 'XYZ', shares=60, price=19)
display(tracker.capital_gains_df)
print("Wash sales:\n"+tracker.washed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2023-02-10,0.0,0


Wash sales:
XYZ: 2023-01-15 40 @ $20.00 >> 2023-02-10 -40 @ $15.00 = $-200.00 ST (26 days) washed on 2023-02-25
XYZ: 2023-01-15 60 @ $20.00 >> 2023-02-10 -60 @ $15.00 = $-300.00 ST (26 days) washed on 2023-03-08

Open positions:
XYZ: 40 @ $17.00 (2023-02-25)  Basis $-200.00 term+26 days
XYZ: 60 @ $19.00 (2023-03-08)  Basis $-300.00 term+26 days


In [24]:
# Make sure we get the same result if we use buy/sell methods
tracker = CapGainsTracker()
tracker.buy(pd.Timestamp('2023-01-15'), 'XYZ', 100, price=20)
tracker.sell(pd.Timestamp('2023-02-10'), 'XYZ', -100, price=15)
tracker.buy(pd.Timestamp('2023-02-25'), 'XYZ', 40, price=17)
tracker.buy(pd.Timestamp('2023-03-08'), 'XYZ', 60, price=19)
display(tracker.capital_gains_df)
print("Wash sales:\n"+tracker.washed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2023-02-10,0.0,0


Wash sales:
XYZ: 2023-01-15 40 @ $20.00 >> 2023-02-10 -40 @ $15.00 = $-200.00 ST (26 days) washed on 2023-02-25
XYZ: 2023-01-15 60 @ $20.00 >> 2023-02-10 -60 @ $15.00 = $-300.00 ST (26 days) washed on 2023-03-08

Open positions:
XYZ: 40 @ $17.00 (2023-02-25)  Basis $-200.00 term+26 days
XYZ: 60 @ $19.00 (2023-03-08)  Basis $-300.00 term+26 days


## Example 1.6
Washed loss and gain in same period.

In [25]:
tracker = CapGainsTracker()
tracker.buy(pd.Timestamp('2023-01-15'), 'XYZ', 100, price=20)
tracker.sell(pd.Timestamp('2023-02-10'), 'XYZ', -100, price=15)
tracker.buy(pd.Timestamp('2023-02-25'), 'XYZ', 100, price=18)
tracker.sell(pd.Timestamp('2023-03-15'), 'XYZ', 100, price=25)
display(tracker.capital_gains_df)
print("Wash sales:\n"+tracker.washed_lots_str)
print("\nClosed positions:\n"+tracker.closed_lots_str)

,ShortTerm,LongTerm
2023-02-10,0.0,0
2023-03-15,200.0,0


Wash sales:
XYZ: 2023-01-15 100 @ $20.00 >> 2023-02-10 -100 @ $15.00 = $-500.00 ST (26 days) washed on 2023-02-25

Closed positions:
XYZ: 2023-01-15 100 @ $20.00 >> 2023-02-10 -100 @ $15.00 = $-500.00 ST (26 days) washed
XYZ: 2023-02-25 100 @ $18.00 >> 2023-03-15 -100 @ $25.00 = $200.00 ST (18+26 days)


## Example 1.7a
Closing trades that are washed by other prior opening trades.

In [26]:
tracker = CapGainsTracker()
tracker.buy(pd.Timestamp('2023-01-15'), 'XYZ', 100, price=20)
tracker.buy(pd.Timestamp('2023-01-16'), 'XYZ', 100, price=20)
tracker.sell(pd.Timestamp('2023-02-10'), 'XYZ', -100, price=15)
tracker.validate()
print("\nClosed positions:\n"+tracker.closed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)


Closed positions:
XYZ: 2023-01-15 100 @ $20.00 >> 2023-02-10 -100 @ $15.00 = $-500.00 ST (26 days) washed

Open positions:
XYZ: 100 @ $20.00 (2023-01-16)  Basis $-500.00 term+26 days


In [27]:
tracker.sell(pd.Timestamp('2023-02-11'), 'XYZ', -50, price=15)
display(tracker.capital_gains_df)
tracker.validate()
print("\nClosed positions:\n"+tracker.closed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)

,ShortTerm,LongTerm
2023-02-11,-500.0,0



Closed positions:
XYZ: 2023-01-15 100 @ $20.00 >> 2023-02-10 -100 @ $15.00 = $-500.00 ST (26 days) washed
XYZ: 2023-01-16 50 @ $20.00 >> 2023-02-11 -50 @ $15.00 = $-500.00 ST (26+26 days)

Open positions:
XYZ: 50 @ $20.00 (2023-01-16)  Basis $-250.00 term+26 days


In [28]:
print(f'Capital gains: ${tracker.capital_gains_df.sum().sum()}')
print(f'Carried gains: ${sum(position.basis_add for position in tracker.open_lots_list)}')
print(f'Price gains on closed trades: ${sum(trade.price_pnl for trade in tracker.closed_lots_list)}')

Capital gains: $-500.0
Carried gains: $-250.0
Price gains on closed trades: $-750.0


## Example 1.7b
Partial washes of closing losses due to previous opens.

In [29]:
tracker = CapGainsTracker()
tracker.buy(pd.Timestamp('2023-01-15'), 'XYZ', 60, price=20)
tracker.buy(pd.Timestamp('2023-01-17'), 'XYZ', 50, price=21)
tracker.buy(pd.Timestamp('2023-01-19'), 'XYZ', 40, price=22)
tracker.sell(pd.Timestamp('2023-02-10'), 'XYZ', shares_to_sell=-20, price=16)
tracker.validate()
# print("\nOpen positions:\n"+tracker.Positions)
tracker.sell(pd.Timestamp('2023-02-11'), 'XYZ', -20, price=15)
# print("\nOpen positions:\n"+tracker.Positions)
tracker.validate()
#print("Wash sales:\n"+tracker.WashTrades)
print("\nClosed positions:\n"+tracker.closed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)
#print(f'\nTotal open shares = {tracker.get_position("XYZ")}')


Closed positions:
XYZ: 2023-01-15 20 @ $20.00 >> 2023-02-10 -20 @ $16.00 = $-80.00 ST (26 days) washed
XYZ: 2023-01-15 20 @ $20.00 >> 2023-02-11 -20 @ $15.00 = $-100.00 ST (27 days) washed

Open positions:
XYZ: 20 @ $20.00 (2023-01-15) 
XYZ: 10 @ $21.00 (2023-01-17) 
XYZ: 40 @ $22.00 (2023-01-19) 
XYZ: 20 @ $21.00 (2023-01-17)  Basis $-80.00 term+26 days
XYZ: 20 @ $21.00 (2023-01-17)  Basis $-100.00 term+27 days


In [30]:
tracker.sell(pd.Timestamp('2023-02-12'), 'XYZ', -40, price=14)
tracker.validate()
#print("Wash sales:\n"+tracker.WashTrades)
print("\nClosed positions:\n"+tracker.closed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)
print(f'\nTotal open shares = {tracker.get_position("XYZ")}')


Closed positions:
XYZ: 2023-01-15 20 @ $20.00 >> 2023-02-10 -20 @ $16.00 = $-80.00 ST (26 days) washed
XYZ: 2023-01-15 20 @ $20.00 >> 2023-02-11 -20 @ $15.00 = $-100.00 ST (27 days) washed
XYZ: 2023-01-15 20 @ $20.00 >> 2023-02-12 -20 @ $14.00 = $-120.00 ST (28 days) washed
XYZ: 2023-01-17 10 @ $21.00 >> 2023-02-12 -10 @ $14.00 = $-130.00 ST (26+28 days) washed
XYZ: 2023-01-19 10 @ $22.00 >> 2023-02-12 -10 @ $14.00 = $-80.00 ST (24 days) washed

Open positions:
XYZ: 10 @ $22.00 (2023-01-19) 
XYZ: 10 @ $21.00 (2023-01-17)  Basis $-40.00 term+26 days
XYZ: 20 @ $21.00 (2023-01-17)  Basis $-100.00 term+27 days
XYZ: 10 @ $22.00 (2023-01-19)  Basis $-60.00 term+28 days
XYZ: 10 @ $22.00 (2023-01-19)  Basis $-130.00 term+54 days
XYZ: 10 @ $21.00 (2023-01-17)  Basis $-120.00 term+50 days

Total open shares = 70.0


## Example 1.7c
Same as 1.7b but selling short.

In [31]:
tracker = CapGainsTracker()
tracker.trade(pd.Timestamp('2023-01-15'), 'XYZ', -60, price=16)
tracker.trade(pd.Timestamp('2023-01-17'), 'XYZ', -50, price=15)
tracker.trade(pd.Timestamp('2023-01-19'), 'XYZ', -40, price=14)
tracker.trade(pd.Timestamp('2023-02-10'), 'XYZ', 20, price=20)
tracker.validate()
tracker.trade(pd.Timestamp('2023-02-11'), 'XYZ', 20, price=21)
tracker.validate()
#print("Wash sales:\n"+tracker.WashTrades)
print("\nClosed positions:\n"+tracker.closed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)
#print(f'\nTotal open shares = {tracker.get_position("XYZ")}')


Closed positions:
XYZ: 2023-01-15 -20 @ $16.00 >> 2023-02-10 20 @ $20.00 = $-80.00 ST (26 days) washed
XYZ: 2023-01-15 -20 @ $16.00 >> 2023-02-11 20 @ $21.00 = $-100.00 ST (27 days) washed

Open positions:
XYZ: -20 @ $16.00 (2023-01-15) 
XYZ: -10 @ $15.00 (2023-01-17) 
XYZ: -40 @ $14.00 (2023-01-19) 
XYZ: -20 @ $15.00 (2023-01-17)  Basis $-80.00 term+26 days
XYZ: -20 @ $15.00 (2023-01-17)  Basis $-100.00 term+27 days


In [32]:
tracker.trade(pd.Timestamp('2023-02-12'), 'XYZ', 40, price=22)
tracker.validate()
#print("Wash sales:\n"+tracker.WashTrades)
print("\nClosed positions:\n"+tracker.closed_lots_str)
print("\nOpen positions:\n"+tracker.open_lots_str)
print(f'\nTotal open shares = {tracker.get_position("XYZ")}')


Closed positions:
XYZ: 2023-01-15 -20 @ $16.00 >> 2023-02-10 20 @ $20.00 = $-80.00 ST (26 days) washed
XYZ: 2023-01-15 -20 @ $16.00 >> 2023-02-11 20 @ $21.00 = $-100.00 ST (27 days) washed
XYZ: 2023-01-15 -20 @ $16.00 >> 2023-02-12 20 @ $22.00 = $-120.00 ST (28 days) washed
XYZ: 2023-01-17 -10 @ $15.00 >> 2023-02-12 10 @ $22.00 = $-130.00 ST (26+28 days) washed
XYZ: 2023-01-19 -10 @ $14.00 >> 2023-02-12 10 @ $22.00 = $-80.00 ST (24 days) washed

Open positions:
XYZ: -10 @ $14.00 (2023-01-19) 
XYZ: -10 @ $15.00 (2023-01-17)  Basis $-40.00 term+26 days
XYZ: -20 @ $15.00 (2023-01-17)  Basis $-100.00 term+27 days
XYZ: -10 @ $14.00 (2023-01-19)  Basis $-60.00 term+28 days
XYZ: -10 @ $14.00 (2023-01-19)  Basis $-130.00 term+54 days
XYZ: -10 @ $15.00 (2023-01-17)  Basis $-120.00 term+50 days

Total open shares = -70.0


# 2. Distribution Tracker
The `DistributionTracker` is a `CapGainsTracker` that adds tax characterization of distributions.  Distributions on equities are most commonly dividends, but they can also consist of capital gains or return of capital.

Taxation of dividends falls into three categories: Some, like dividends paid from municipal bond funds, are Tax-Exempt.  Regular dividends are taxed as ordinary income unless they are Qualified.  Qualified dividends are taxed as long-term capital gains.  A Regular dividend is "Qualified" if, among other requirements, the owner satisfies the **Holding period** described on p.28 of IRS Publication 550.  For most eligible dividends this requirement is that the owner holds the stock for more than 60 calendar days of the 121 calendar days centered on the stock's ex-dividend date.  (Due to this awkward rule, we can't be sure of the characterization of regular dividends until up to 60 days after the ex-date.  And for some preferred stock dividends it's worse: the Holding period requirement is *more than 90 days during the 181-day period centered on the ex-date*.)  Primary reference for this is [IRC Sec. 1(h)(11)](https://www.law.cornell.edu/uscode/text/26/1#h_11).

The *ex-dividend date* is the date on which one must own the share to receive the dividend.  In effect, before the market opens on the ex-date the entity paying the dividend records who owns each share, and the owner at that moment is who will receive the dividend.  (Actual payment of the dividend typically takes place a few weeks later.)  All else equal, when the market opens for trading on the ex-date the stock will begin trading at a price reduced by the amount of the dividend.  The ex-date is declared in advance, along with the amount of the dividend.

**Example:** Company ABC declares a $1 dividend with an ex-date of Friday, March 15.  If you want to receive that dividend then you need to buy and hold shares by the close of the previous market day (Thursday, March 14).  And if the share price at the close of March 14 is $10 then you would expect the shares to trade at $9 when the market opens on March 15.

**Shorts:** If you hold a short position on the ex-date then the asset lender will require you to make a *payment-in-lieu* (PIL) of the dividends.  Payments in lieu of dividends qualify as an *Interest Expense* if the short position is held open for more than 45 days.  Otherwise the PIL is added to the cost basis of the short position.

The `DistributionTracker` requires distribution data, which it takes in the form of a DataFrame structured as follows:
* Rows are MultiIndexed by `Ticker`, `Date`, and `Type`.
* The value column is the amount of the `Distribution`.
* `Date` is the ex-date of the distribution.
* `Type` is a single letter describing the type of distribution and must be one of the following:
    * [D]ividend (regular)
    * [P]referred ("due to periods totaling more than 366 days")
    * [N]ot qualified (i.e., ineligible to be qualified regardless of holding period)
    * [E]xempt (tax-exempt)
    * [R]eturn of Capital (also tax-exempt)
    * [L]ong-Term Capital Gain
    * [S]hort-Term Capital Gain

Following is a sample of distribution data prepared this way.  Note that `NEA` is a municipal bond fund, so its dividends are exempt.  `RA` is a regular bond fund, so its dividends are regular:

In [33]:
import pandas as pd
distribution_data = pd.read_csv(r'tests/DistributionData.csv',
                                index_col=[0, 1, 2], parse_dates=[1]).sort_index()
display(distribution_data[:5])  # First stock is NEA
display(distribution_data[distribution_data.index.get_level_values(0)=='RA'][:5])

Distribution
Ticker Date       Type              
NEA    2022-01-13 E         0.058490
       2022-02-14 E         0.058553
       2022-03-14 E         0.058494
       2022-04-13 E         0.052452
       2022-05-12 E         0.052553

Distribution
Ticker Date       Type              
RA     2022-01-18 D         0.199041
       2022-02-08 D         0.198933
       2022-03-15 D         0.199014
       2022-04-12 D         0.199019
       2022-05-10 D         0.199019

## Example 2.1
Buy and hold for exactly 60 days a single share of a stock that pays regular dividends.  Dividends will not be qualified.

In [34]:
from tax_tracker import DistributionTracker
tracker = DistributionTracker(distribution_data)
buy_date = pd.Timestamp('2022-01-05')
tracker.trade(buy_date, 'RA', shares=1, price=21)
sell_date = buy_date + pd.Timedelta(days=60)
tracker.trade(sell_date, 'RA', shares=-1, price=21)
tracker.validate()
tracker.dividends_df

,Regular
2022-01-18,0.199041
2022-02-08,0.198933


Hold for more than 60 days and regular dividends are qualified:

In [35]:
from tax_tracker import DistributionTracker
tracker = DistributionTracker(distribution_data)
buy_date = pd.Timestamp('2022-01-05')
tracker.trade(buy_date, 'RA', shares=1, price=21)
sell_date = buy_date + pd.Timedelta(days=61)
tracker.trade(sell_date, 'RA', shares=-1, price=21)
tracker.validate()
tracker.dividends_df

,Qualified
2022-01-18,0.199041
2022-02-08,0.198933


Now try both together, and throw in an exempt position while we're at it:

In [36]:
from tax_tracker import DistributionTracker
tracker = DistributionTracker(distribution_data)
buy_date = pd.Timestamp('2022-01-05')
tracker.trade(buy_date, 'RA', shares=2, price=21)
tracker.trade(buy_date, 'NEA', shares=1, price=11)
sell_date = buy_date + pd.Timedelta(days=60)
tracker.trade(sell_date, 'RA', shares=-1, price=21)
tracker.trade(sell_date+pd.Timedelta(days=1), 'RA', shares=-1, price=21)
tracker.validate()
tracker.dividends_df

,Regular,Qualified,Exempt
2022-01-13,0.000000,0.000000,0.058490
2022-01-18,0.199041,0.199041,0.000000
2022-02-08,0.198933,0.198933,0.000000
2022-02-14,0.000000,0.000000,0.058553


## Example 2.2
While short a stock, we make payments-in-lieu (PIL) of dividends.  Initially accounted as capital gains, these become expenses if the short is held for more than 45 days.

In this example we'll short 2 shares, covering 1 share after 45 days.  At that point PIL on both shares are accounted to ShortTerm capital gains.

When we cover the second share after more than 45 days its PIL will instead appear as an interest Expense.

In [37]:
tracker = DistributionTracker(distribution_data)
start_date = pd.Timestamp('2022-01-05')
tracker.trade(start_date, 'NEA', shares=-2, price=11)
cover_date = start_date + pd.Timedelta(days=45)
tracker.trade(cover_date, 'NEA', 1, price=11)
display(tracker.dividends_df)
display(tracker.taxable_cashflows_df)

,PayInLieu
2022-01-13,-0.116980
2022-02-14,-0.117106


,ShortTerm
2022-01-13,-0.116980
2022-02-14,-0.117106


In [38]:
tracker.trade(cover_date+pd.Timedelta(days=1), 'NEA', 1, price=11)
tracker.validate()
print('Now covering the other share after more than 45 days, PIL account to Expenses:')
display(tracker.dividends_df)
display(tracker.taxable_cashflows_df)
for divs in tracker.daily_dividends.values():
    assert divs['PayInLieu'] == divs['Expense'], 'PIL correct'

Now covering the other share after more than 45 days, PIL account to Expenses:


,PayInLieu,Expense
2022-01-13,-0.058490,-0.058490
2022-02-14,-0.058553,-0.058553


,Expense,ShortTerm
2022-01-13,-0.058490,-0.058490
2022-02-14,-0.058553,-0.058553


## Example 2.3
Buy and hold one share for more than 60 days.  Later buy a second share and continue holding both for more than 60 days.  All their dividends should be Qualified.

In [39]:
tracker = DistributionTracker(distribution_data)
start_date = pd.Timestamp('2022-01-05')
tracker.buy(start_date, 'RA', shares=1, price=21)
second_buy_date = start_date + pd.Timedelta(days=60)
tracker.buy(second_buy_date, 'RA', shares=1, price=21)
print(f'Second share bought on {second_buy_date.date()}')
# Run the tracker until June 1, 2022
tracker.BoD(pd.Timestamp('2022-06-01'))
tracker.validate()
tracker.dividends_df

Second share bought on 2022-03-06


,Qualified
2022-01-18,0.199041
2022-02-08,0.198933
2022-03-15,0.398029
2022-04-12,0.398037
2022-05-10,0.398037


But if we sell a share within 60 days of buying the second share, then its dividends are not Qualified:

In [40]:
tracker = DistributionTracker(distribution_data)
start_date = pd.Timestamp('2022-01-05')
tracker.buy(start_date, 'RA', shares=1, price=21)
second_buy_date = start_date + pd.Timedelta(days=60)
tracker.buy(second_buy_date, 'RA', shares=1, price=21)
tracker.sell(second_buy_date + pd.Timedelta(days=60), 'RA', shares_to_sell=1, price=21)
# Run the tracker until July 1, 2022
tracker.BoD(pd.Timestamp('2022-07-01'))
tracker.validate()
tracker.dividends_df[['Regular', 'Qualified']]

,Regular,Qualified
2022-01-18,0.000000,0.199041
2022-02-08,0.000000,0.198933
2022-03-15,0.199014,0.199014
2022-04-12,0.199019,0.199019
2022-05-10,0.000000,0.199019
2022-06-14,0.000000,0.199003


## Example 2.4
Buy the day before a dividend; sell some number of days after.  Which dividends are qualified depends on the timing of dividends and the number of days we hold each time.

In [41]:
tracker = DistributionTracker(distribution_data)
start_date = pd.Timestamp('2022-01-14')
# Get 10 months of ex-dividend dates for RA
ex_dates = distribution_data.loc[pd.IndexSlice['RA',
    (start_date):(start_date+pd.DateOffset(months=10)), :]].index.get_level_values(1)
for date in ex_dates:
    # Buy the day before each ex-date, so we get the dividend; then sell some days after.
    buy_date = date + pd.Timedelta(days=-1)
    #print(f'Buying on {buy_date.date()}')
    tracker.buy(buy_date, 'RA', shares=1, price=20)
    tracker.sell(date + pd.Timedelta(days=14), 'RA', 1, price=20.5)
    tracker.validate()
print('\nSummary cashflows for tax calculations:\n'
      + pd.DataFrame(tracker.taxable_cashflows_df.sum()).T.to_string(index=False))


Summary cashflows for tax calculations:
 Regular  Qualified  ShortTerm
 1.59203    0.59699        5.5


Did we get it right?  We'll add the calculation of holding days corresponding to each dividend:

In [42]:
period = 60
df = tracker.dividends_df.copy()
df['HoldDays'] = .0
for date in df.index:
    df.loc[date, 'HoldDays'] = tracker.holdings.shift() \
        .loc[(date - pd.Timedelta(days=period)):(date + pd.Timedelta(days=period)),
             "RA"].nlargest(period+1).sum()
df

,Regular,Qualified,HoldDays
2022-01-18,0.199041,0.000000,35.0
2022-02-08,0.198933,0.000000,45.0
2022-03-15,0.000000,0.199014,61.0
2022-04-12,0.199019,0.000000,57.0
2022-05-10,0.199019,0.000000,60.0
2022-06-14,0.000000,0.199003,61.0
2022-07-12,0.198963,0.000000,57.0
2022-08-09,0.199016,0.000000,60.0
2022-09-13,0.000000,0.198973,61.0
2022-10-11,0.199064,0.000000,57.0


## Example 2.5
We'll create a synthetic stock `ABC` that pays a preferred dividend that is 2/3 qualified and "due to periods totaling more than 366 days," which means that if held more than 90 days during the 181 days centered on the ex-date then the qualifiable portion becomes qualified.

In [43]:
# We'll buy stock on Jan 2, 2025.
start_date = pd.Timestamp('2025-01-02')
qualified_date = start_date + pd.Timedelta(days=91)
print(f'Preferred qualified_date: {qualified_date}')
syn_data = {
    'Ticker': ['ABC', 'ABC'],
    'Date': ['2025-02-01', '2025-02-01'],
    'Type': ['P', 'N'],
    'Distribution': [0.10, 0.05]
}
test_data = pd.DataFrame(syn_data)
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data.set_index(['Ticker', 'Date', 'Type'], inplace=True)
test_data

Preferred qualified_date: 2025-04-03 00:00:00


Distribution
Ticker Date       Type              
ABC    2025-02-01 P             0.10
                  N             0.05

In [44]:
tracker = DistributionTracker(test_data)
tracker.buy(start_date, 'ABC', shares=1, price=10)
pre_qualification_date = qualified_date - pd.Timedelta(days=1)
tracker.BoD(pre_qualification_date)
print(f'Dividends as of {pre_qualification_date.date()}, just before qualification date:')
display(tracker.dividends_df)
tracker.BoD(qualified_date)
tracker.validate()
print('Dividends on qualification date:')
display(tracker.dividends_df)

Dividends as of 2025-04-02, just before qualification date:


,Regular
2025-02-01,0.15


Dividends on qualification date:


,Regular,Qualified
2025-02-01,0.05,0.1


## Example 2.6
We'll reproduce the example [here](https://www.fidelity.com/tax-information/tax-topics/qualified-dividends):

You purchased 10,000 shares of XYZ fund on April 27 of the tax year. You sold 2,000 of those shares on June 15, but continue to hold (unhedged at all times) the remaining 8,000 shares. The ex-dividend date for XYZ fund was May 2.  A dividend of $0.18 per share was paid but only 50% of that dividend ($0.09 per share) was reported as a qualified dividend.

Therefore, during the 121-day window, you held 2,000 shares for 49 days (from April 28 through June 15) and 8,000 shares for at least 61 days (from April 28 through July 1).

The dividend income from the 2,000 shares held 49 days would not be qualified dividend income. The dividend income from the 8,000 shares held at least 61 days should be qualified dividend income, but only half of the dividend can be qualified so only $720 of dividends end up qualified for tax purposes.

In [45]:
tracker = DistributionTracker(None)
tracker.trade(pd.Timestamp('2025-04-27'), 'XYZ', 10_000, price=1)
tracker.receive_distribution(pd.Timestamp('2025-05-02'), 'XYZ', 0.09, 'D')
tracker.receive_distribution(pd.Timestamp('2025-05-02'), 'XYZ', 0.09, 'N')
#display(tracker.Dividends)
tracker.trade(pd.Timestamp('2025-06-15'), 'XYZ', -2_000, price=1)
tracker.BoD(pd.Timestamp('2025-07-01'))
display(tracker.dividends_df)

,Regular,Qualified
2025-05-02,1080.0,720.0


## Example 2.7
Complete test coverage by passing capital gains distributions and return of capital.

In [46]:
start_date = pd.Timestamp('2025-01-02')
syn_data = {
    'Ticker': ['ABC', 'ABC', 'ABC'],
    'Date': ['2025-02-01', '2025-02-01', '2025-02-01'],
    'Type': ['L', 'S', 'R'],
    'Distribution': [0.10, 0.05, 0.03]
}
test_data = pd.DataFrame(syn_data)
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data.set_index(['Ticker', 'Date', 'Type'], inplace=True)
tracker = DistributionTracker(test_data)
tracker.buy(start_date, 'ABC', shares=1, price=10)
after_div_date = start_date + pd.DateOffset(months=1)
tracker.BoD(after_div_date)
tracker.taxable_cashflows_df

,Exempt,ShortTerm,LongTerm
2025-02-01,0.03,0.05,0.1


# 3. PNL Tracker
`PNLtracker` is a `DistributionTracker` that adds a record of daily P&L and NAV.  To do this it requires daily price data in the form of a DataFrame structured as follows:
* Rows are MultiIndexed by [`Ticker`, `Date`]
* Column `Px` is the split-adjusted closing price.
* Optional column `AdjPx` is a total-return series that incorporates distributions.  If provided it will be used to error-check calculations.

(Formula for `AdjPx[t] = AdjPx[t-1] * (Px[t] + Distributions[t]) / Px[t-1]`)

The price data also allow us to call `trade()` without specifying a price, in which case the tracker will assume that the trades occurred at the closing price on the date indicated.

Following is sample price data:

In [47]:
import pandas as pd
price_data = pd.read_csv(r'tests/PriceData.csv',
            index_col=[0, 1], parse_dates=[1]).sort_index()
display(price_data[:5])

Px    AdjPx
Ticker Date                      
NEA    2022-01-03  15.54  16.2702
       2022-01-04  15.47  16.1969
       2022-01-05  15.17  15.8828
       2022-01-06  14.99  15.6943
       2022-01-07  15.09  15.7990

## Example 3.1
Hold `RA` for one year.  Dividends will be qualified and gains will be long-term.

In [48]:
import pandas as pd
from tax_tracker import PNLtracker
price_data = pd.read_csv(r'tests/PriceData.csv',
            index_col=[0, 1], parse_dates=[1]).sort_index()
distribution_data = pd.read_csv(r'tests/DistributionData.csv',
            index_col=[0, 1, 2], parse_dates=[1]).sort_index()

tracker = PNLtracker(price_data, distribution_data)
start_date = pd.Timestamp('2022-01-05')
end_date = start_date + pd.DateOffset(days=367)
tracker.BoD(start_date)
tracker.trade_dollars(start_date, 'RA', dollars=1_000)
tracker.BoD(end_date)
tracker.validate()
tracker.close(end_date, 'RA')
tracker.EoD(end_date)
tracker.validate()
display(divs := tracker.dividends_df.T)
display(gains := tracker.capital_gains_df)
pnl = tracker.daily_pnl_df
print('PNL = Dividends + Capital gains:'
      f'\n{pnl.sum().PNL:.2f} = {divs.sum().sum():.2f} + {gains.sum().sum():.2f}')

,2022-01-18,2022-02-08,2022-03-15,2022-04-12,2022-05-10,2022-06-14,2022-07-12,2022-08-09,2022-09-13,2022-10-11,2022-11-08,2022-12-13
Qualified,9.388737,9.383619,9.387466,9.387675,9.387667,9.38694,9.385065,9.387546,9.385519,9.389825,9.385633,9.388311


,ShortTerm,LongTerm
2023-01-07,0,-203.773585


PNL = Dividends + Capital gains:
-91.13 = 112.64 + -203.77


## Example 3.2

Let's repeat the buy-before-dividend strategy from Example 2.4, then summarize is performance for tax analysis.

In [49]:
tracker = PNLtracker(price_data, distribution_data)
start_date = pd.Timestamp('2022-01-14')
# Get the next 8 months of ex-dividend dates for RA
ex_dates = distribution_data.loc[pd.IndexSlice['RA',
        (start_date):(start_date+pd.DateOffset(months=8)), :]].index.get_level_values(1)
for date in ex_dates:
    # Buy the day before each ex-date, so we get the dividend
    buy_date = date + pd.Timedelta(days=-1)
    tracker.buy(buy_date, 'RA', shares=1)
    tracker.sell(date + pd.Timedelta(days=14), 'RA', 1)
taxable_cashflows = tracker.taxable_cashflows_df
display(taxable_cashflows)
tracker.validate()
print(tracker.closed_lots_str)
print('\nSummary cashflows for tax calculations:\n'
      + pd.DataFrame(taxable_cashflows.sum()).T.to_string(index=False))

,Regular,Qualified,ShortTerm
2022-01-18,0.199041,0.000000,0.00
2022-02-08,0.198933,0.000000,0.00
2022-03-15,0.000000,0.199014,0.00
2022-04-12,0.199019,0.000000,0.00
2022-05-10,0.199019,0.000000,0.00
2022-06-14,0.000000,0.199003,0.00
2022-07-12,0.198963,0.000000,0.00
2022-08-09,0.199016,0.000000,0.00
2022-09-13,0.198973,0.000000,0.00
2022-09-27,0.000000,0.000000,-7.42


RA: 2022-01-17 1 @ $21.83 >> 2022-02-01 -1 @ $21.30 = $-0.53 ST (15 days) washed
RA: 2022-02-07 1 @ $21.14 >> 2022-02-22 -1 @ $20.07 = $-1.60 ST (15+15 days) washed
RA: 2022-03-14 1 @ $20.74 >> 2022-03-29 -1 @ $20.62 = $-1.72 ST (15+30 days) washed
RA: 2022-04-11 1 @ $20.92 >> 2022-04-26 -1 @ $20.57 = $-2.07 ST (15+45 days) washed
RA: 2022-05-09 1 @ $20.72 >> 2022-05-24 -1 @ $18.79 = $-4.00 ST (15+60 days) washed
RA: 2022-06-13 1 @ $18.40 >> 2022-06-28 -1 @ $18.30 = $-4.10 ST (15+75 days) washed
RA: 2022-07-11 1 @ $19.34 >> 2022-07-26 -1 @ $19.03 = $-4.41 ST (15+90 days) washed
RA: 2022-08-08 1 @ $20.13 >> 2022-08-23 -1 @ $19.38 = $-5.16 ST (15+105 days) washed
RA: 2022-09-12 1 @ $19.41 >> 2022-09-27 -1 @ $17.15 = $-7.42 ST (15+120 days)

Summary cashflows for tax calculations:
 Regular  Qualified  ShortTerm
1.392964   0.398017      -7.42
